# 1. Interpretation and significance

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 = 873 + 0.0012 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 ^2 − 223.57 ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

1. we have to check if those coefficients are statistically significant

2. If coefficients are statistically significant we can say the following:

- 873 - it's a bias
- 𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 has a quadratic dependence on income. 
- families with children pay 223 less


# 2. Weather model

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.decomposition import PCA 
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

In [24]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [25]:
weather_df.head(2)

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.


In [26]:
weather_df['target'] = weather_df['apparenttemperature'] - weather_df['temperature']


In [28]:
# Y is the target variable
Y = weather_df['apparenttemperature'] - weather_df['temperature']
# X is the feature set
X = weather_df[['humidity','windspeed']]
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 14 Aug 2019   Prob (F-statistic):               0.00
Time:                        19:24:07   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The estimated coefficients statistically significant (p-values = 0)

target = 2.4381 - 3.0292 humidity - 0.1193 windspeed

According to the estimated model, 1 point increase in the humidity results in 3.0292 points decrease in the target. 1 point increase in the windspeed is associated with 0.1193 point decrease in our target.


In [29]:
Y = weather_df['target']
weather_df['Hum_Wind'] = weather_df['humidity'] * weather_df['windspeed']
X = weather_df[['humidity', 'windspeed', 'Hum_Wind']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

# We print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 14 Aug 2019   Prob (F-statistic):               0.00
Time:                        19:28:11   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0839      0.033      2.511      0.0

1) coefficients are still statistically significant (since p-value < 0.05)

2) after including the intereaction term, signs of humidity and windspeed changed from negative to the positive

3) According to the model, 1 point increase in the humidity and windspeed results in 0.18 and 0.09 point increases in the target respectively.

4) According to the model, the coefficient of the interaction term is -0.30. We can interpret it as follows. Given a windspeed level, 1 point increase in humidity results in 0.18 - 0.30 X windspeed point increase in the target. This means that the increase in the target is lower for high values of windspeed than for low values of windspeed. So, the windspeed mitigates the effect of humidity increase on the target. Similarly for a given humidity level, 1 point increase in the windspeed results in 0.09 - 0.30 X humidity point increase in the target. So, the humidity also mitigates the effect of windspeed on the target.

# 3. House prices model

In [31]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [32]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [33]:
houseprices.head(2)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500


In [41]:
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
houseprices['salecondition']= label_encoder.fit_transform(houseprices['salecondition']) 

houseprices['flSF'] = houseprices['firstflrsf'] + houseprices['secondflrsf']
Y = houseprices['saleprice']
X = houseprices[['salecondition', 'overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'flSF']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     676.5
Date:                Wed, 14 Aug 2019   Prob (F-statistic):               0.00
Time:                        20:20:45   Log-Likelihood:                -17486.
No. Observations:                1460   AIC:                         3.499e+04
Df Residuals:                    1452   BIC:                         3.503e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.108e+05   5352.962    -20.695      0.000   -1.21e+05      -1e+05
salecondition  4164.4378    942.121      4.420      0.000    2316.375    6012.501
overallqual    2.296e+04   1071.927     21.424      0.000    2.09e+04    2.51e+04
grlivarea        -3.0007     21.016     -0.143      0.886     -44.226      38.224
garagecars      1.36e+04   3010.050      4.518      0.000    7694.012    1.95e+04
garagearea       15.8501     10.392      1.525      0.127      -4.535      36.235
totalbsmtsf      31.2414      2.883     10.836      0.000      25.586      36.897
flSF             50.2872     21.401      2.350      0.019       8.307      92.267
==============================================================================
Omnibus:                      465.395   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            48725.337
Skew:                          -0.422   Prob(JB):                         0.00
Kurtosis:                      31.289   Cond. No.                     1.37e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.37e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

grlivarea and garagearea are statistically insignificant. Hence we will exclude those.

In [42]:
Y = houseprices['saleprice']
X = houseprices[['salecondition', 'overallqual', 'garagecars', 'totalbsmtsf', 'flSF']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     946.4
Date:                Wed, 14 Aug 2019   Prob (F-statistic):               0.00
Time:                        20:23:57   Log-Likelihood:                -17487.
No. Observations:                1460   AIC:                         3.499e+04
Df Residuals:                    1454   BIC:                         3.502e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.111e+05   5338.868    -20.817      0.000   -1.22e+05   -1.01e+05
salecondition  4224.3248    940.198      4.493      0.000    2380.035    6068.614
overallqual     2.29e+04   1071.239     21.378      0.000    2.08e+04     2.5e+04
garagecars     1.735e+04   1743.766      9.949      0.000    1.39e+04    2.08e+04
totalbsmtsf      32.1931      2.815     11.435      0.000      26.670      37.716
flSF             47.5299      2.512     18.919      0.000      42.602      52.458
==============================================================================
Omnibus:                      447.827   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44460.577
Skew:                          -0.357   Prob(JB):                         0.00
Kurtosis:                      30.025   Cond. No.                     1.03e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.03e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now we have all variables statistically significant.  According to the estimation results:
- 1 point increase in salecondition results in \$4224 increase in sale price.
- 1 point increase in overallqual results in \$22900 increase in sale price.
- 1 point increase in garagecars results in \$17350 increase in sale price.
- 1 point increase in totalbsmtsf results in \$32 increase in sale price.
- 1 point increase in flSF results in \$47 increase in sale price.

Considering the results, the most prominent factors affecting the sale price seem to be the overallqual and the garagecars.